In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from IPython.display import display, clear_output, Image
from PIL import Image as PILImage
import io

# Load the ASL model
classifier = load_model('asl_model.h5')

# Mapping between class indices and alphabet labels
alphabet_mapping = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 
    'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'SPACE', 
    'DELETE', 'NOTHING'
]

# Start video capture
cap = cv2.VideoCapture(0)

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        frame = cv2.flip(frame, 1)

        # Define region of interest (ROI) on the right side of the screen
        roi = frame[100:400, 700:1000]  # Adjust coordinates as needed
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        roi = roi.reshape(1, 28, 28, 1) 
        roi = roi / 255.0  # Normalize the image

        # Predict the ASL alphabet
        probabilities = classifier.predict(roi)
        predicted_class = np.argmax(probabilities)
        alphabet_label = alphabet_mapping[predicted_class]

        # Display the predicted label on the frame
        cv2.rectangle(frame, (700, 100), (1000, 400), (255, 0, 0), 5)  # Highlight ROI
        cv2.putText(frame, alphabet_label, (700, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)

        # Convert the frame to RGB format and display it in Jupyter
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = PILImage.fromarray(frame_rgb)

        # Display the image in Jupyter
        clear_output(wait=True)  # Clear the previous output
        display(pil_img)

except KeyboardInterrupt:
    print("Stopped.")

finally:
    # Release the video capture object when done
    cap.release()
    cv2.destroyAllWindows()
